In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
data_train = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
data_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(data_train, batch_size=128, shuffle=True)
test_loader = DataLoader(data_test, batch_size=128, shuffle=False)


100%|██████████| 169M/169M [02:16<00:00, 1.24MB/s] 


Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4))
        )

        self.fc = nn.Sequential(
            nn.Linear(256 * 4 * 4, 512), nn.ReLU(),
            nn.Linear(512, 100)
        )
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight)
                with torch.no_grad():
                    m.weight *= torch.sqrt(torch.tensor(2.))

    def forward(self, x):
        x = self.layers(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [6]:
def train(model, epochs=10):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        correct, total = 0, 0
        model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f} | Accuracy: {acc:.2f}%")

# Run the model
model = CustomCNN()
train(model)


Epoch 1 | Loss: 3.6637 | Accuracy: 24.95%
Epoch 2 | Loss: 2.7087 | Accuracy: 35.90%
Epoch 3 | Loss: 2.1578 | Accuracy: 41.30%
Epoch 4 | Loss: 1.7131 | Accuracy: 45.43%
Epoch 5 | Loss: 1.3098 | Accuracy: 47.18%
Epoch 6 | Loss: 0.9256 | Accuracy: 46.05%
Epoch 7 | Loss: 0.6000 | Accuracy: 45.78%
Epoch 8 | Loss: 0.3908 | Accuracy: 45.45%
Epoch 9 | Loss: 0.2932 | Accuracy: 44.07%
Epoch 10 | Loss: 0.2437 | Accuracy: 44.13%
